# Cell 1: Imports & Configuration

In [ ]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image, ImageOps # ImageOps is critical for fixing rotation
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Configuration ---
# Path to your saved model file
MODEL_PATH = r'D:\Machine Learning project\Natural Images Project\PythonCode\MobileNetV2_classifier.keras'

# Path to the new image you want to test
# You can change this path to test different images!
NEW_IMAGE_PATH = r"D:\Machine Learning project\Natural Images Project\cars-dataset\Nissan-Zamiad\Nissan-Zamiad (3).jpg"

# Target size must match what you used during training (256x256)
TARGET_SIZE = (256, 256)
PADDING_COLOR = (0, 0, 0) # Black padding

# --- Load Model & Define Classes ---
print(f"Loading model from: {MODEL_PATH}")
loaded_model = keras.models.load_model(MODEL_PATH)

# IMPORTANT: These must match the alphabetical order of folders in your training set exactly.
class_names = ['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']
print(f"Model loaded. Class labels: {class_names}")

# Cell 2: Preprocessing Function

In [ ]:
def resize_with_true_padding(img, target_size, padding_color=(0, 0, 0)):
    """
    Resizes an image to fit within the target size by adding padding.
    Maintains the original aspect ratio to prevent distortion.
    """
    # 1. Convert to RGB to handle PNGs with transparency or Grayscale images
    if img.mode == 'RGBA':
        # Create a solid background using the padding color
        bg = Image.new('RGB', img.size, padding_color)
        # Paste the image on top using alpha channel as mask
        bg.paste(img, (0, 0), img.split()[-1])
        img = bg
    elif img.mode != 'RGB':
        img = img.convert('RGB')

    # 2. Resize the image (thumbnail) so it fits inside the target box
    # This maintains the aspect ratio (doesn't stretch the image)
    img.thumbnail(target_size, Image.Resampling.LANCZOS)
    
    # 3. Create a new blank background image
    new_img = Image.new("RGB", target_size, padding_color)
    
    # 4. Paste the resized image into the center of the background
    paste_x = (target_size[0] - img.width) // 2
    paste_y = (target_size[1] - img.height) // 2
    new_img.paste(img, (paste_x, paste_y))
    
    return new_img

# Cell 3: Prediction & Visualization

In [ ]:
# --- 1. Load the Image ---
if not os.path.exists(NEW_IMAGE_PATH):
    print(f"Error: Image not found at {NEW_IMAGE_PATH}")
else:
    print(f"Processing image: {NEW_IMAGE_PATH}")
    img = Image.open(NEW_IMAGE_PATH)

    # --- 2. Fix Orientation (The Critical Fix) ---
    # Phones often save images with EXIF rotation tags. 
    # This physically rotates the pixels so the model sees it upright.
    img = ImageOps.exif_transpose(img) 

    # --- 3. Preprocess ---
    # Apply the exact same padding logic used during training
    img_processed = resize_with_true_padding(img, TARGET_SIZE, PADDING_COLOR)

    # --- 4. Prepare for Model ---
    # Convert PIL image to a NumPy array
    img_array = tf.keras.utils.img_to_array(img_processed)
    # Add a batch dimension (Models expect a batch of images, even if it's just 1)
    # Shape becomes: (1, 256, 256, 3)
    img_batch = np.expand_dims(img_array, axis=0) 

    # --- 5. Predict ---
    prediction_scores = loaded_model.predict(img_batch, verbose=0)

    # --- 6. Interpret Results ---
    # Find the index with the highest score
    predicted_class_index = np.argmax(prediction_scores[0])
    # Get the name corresponding to that index
    predicted_class_name = class_names[predicted_class_index]
    # Get the confidence score (probability)
    confidence_score = np.max(prediction_scores[0])

    # --- 7. Display ---
    plt.figure(figsize=(6, 6))
    plt.imshow(img_array.astype("uint8")) 
    plt.title(f"Prediction: {predicted_class_name}\nConfidence: {confidence_score * 100:.2f}%")
    plt.axis('off')
    plt.show()